# Quick Start: Azure AI Foundry SDK + MCP Client

### Step 1: Import the needed packages.

In [1]:
# --- Standard Library Imports ---
import os
import sys
from dotenv import load_dotenv

# --- Azure AI Foundry SDK for AI agents and services ---
from azure.ai.agents.models import CodeInterpreterTool, FilePurpose, FileSearchTool, ListSortOrder
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Load the environment variables from a .env file
load_dotenv()
print(f"Project Endpoint: {os.getenv('PROJECT_ENDPOINT')}")
print(f"Model Deployment Name: {os.getenv('MODEL_DEPLOYMENT_NAME')}")

# Check the current working Python execution environment
print(f"Current Python executable: {sys.executable}")

Project Endpoint: https://*****.services.ai.azure.com/api/projects/*****
Model Deployment Name: gpt-4o
Current Python executable: c:\Users\Yu-Hong (Frank) Lin\Repos\azure-ai-agent\.venv\Scripts\python.exe


### Step 2: create AI Project Client and create an Azure AI Foundry Agent
##### *server_labelis* a unique name you provide for the MCP server within the same Foundry Agent
##### *server_url* is the url of the MCP server
##### *require_approval* supports never

In [2]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"], 
        name="04_my github agent", 
        instructions="You are a helpful assistant. Use the tools provided to answer the user's questions. Be sure to cite your sources.",
        tools= [
            {
                "type": "mcp",
                "server_label": "repo",
                "server_url": "https://gitmcp.io/ownway22/azure-ai-agent",
            }
        ],
        tool_resources=None
    )
    print(f"Created agent, agent ID: {agent.id}")


Created agent, agent ID: asst_R8PSdm1FvNvV2Wzbr1XPK0Rt


### Step 3: Create a Thread, Message and Run
##### Within the Run, you can pass custom headers and use the server_label you provided to map to the specific MCP server.

In [3]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

with project_client:
    thread = project_client.agents.threads.create()
    print(f"Created thread, thread ID: {thread.id}")

    message = project_client.agents.messages.create(
        thread_id=thread.id, role="user", content="who is the author of this repository?",
    )
    print(f"Created message, message ID: {message.id}")

    run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)

Created thread, thread ID: thread_4DyKRGA9alO2zcvjf0aAE4sP
Created message, message ID: msg_xw0yDmXtzfUjqTPfskt122pS


### Step 4: Execute the Run and retrieve Message
##### You can use run_step to get more details on tool inputs and tool outputs. Once the run is complete, you can retrieve message back from the Foundry Agent.

In [4]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

with project_client:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

    run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Run step: {step.id}, status: {step.status}, type: {step.type}")
        if step.type == "tool_calls":
            print(f"Tool call details:")
            for tool_call in step.step_details.tool_calls:
                print(json.dumps(tool_call.as_dict(), indent=2))

    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for data_point in messages:
        last_message_content = data_point.content[-1]
        if isinstance(last_message_content, MessageTextContent):
            print(f"{data_point.role}: {last_message_content.text.value}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED
Run step: step_oHTbX6wIwBxKJlu5O1mae40T, status: RunStepStatus.COMPLETED, type: RunStepType.MESSAGE_CREATION
MessageRole.USER: who is the author of this repository?
MessageRole.AGENT: The author of the repository "ownway22/azure-ai-agent" is the GitHub user **ownway22**.


### Step 5: Clean up

In [ ]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

with project_client:
    project_client.agents.delete_agent(agent.id)
    print(f"Deleted agent, agent ID: {agent.id}")

---

##### Reference: [Announcing Model Context Protocol Support (preview) in Azure AI Foundry Agent Service | Azure AI Foundry Blog](https://devblogs.microsoft.com/foundry/announcing-model-context-protocol-support-preview-in-azure-ai-foundry-agent-service/)